# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Assignment 3: Gradient Descent and Maximum Likelihood

In [8]:
require './assignment_lib'

false

## Question 1.1 (5 points)

Let's implement a test function for the gradient descent optimizer, a 3D simple parabola. All gradient-optimized trainers are implemented as a objective function. The follow the same basic pattern:

```ruby
class MyGradientLearnableModel
  def func x, w
    #Returns the value of the objective function, 
    #  summing across all examples in x
  end
  def grad x, w
    #Returns a hash of derivative values for each variable in w,
    # gradient is summed across all examples in x
    dw = {"0" => (w["0"] - 1), "1" => (w["1"] - 2)}
  end
  def adjust w
    # Applies any problem-specific alterations to w
  end
end
```

Now, let's implement a Parabola objective function which does not depend on the data at all. It is defined as follows:

### $L(w) = \frac{1}{2}\left( \left(w_{0} - 1\right)^2 + \left(w_{1} - 2\right)^2 \right)$

Implement the ```func``` method for the loss function, $L(w)$. 

**Note**: The data, i.e., ```x``` is not actually used in this objective function. Other objective functions may use ```x```. 

In [9]:
class ParabolaObjective
  def func x, w
    sum=1.0/2.0*((w["0"]-1)**2+(w["1"]-2)**2)
    return sum
  end
  def adjust w
    return w
  end
end

:adjust

In [10]:
### TESTS ###
t1 = ParabolaObjective.new
assert_in_delta(0.0, t1.func([], {"0" => 1.0, "1" => 2.0}), 1e-3)
assert_in_delta(0.5, t1.func([], {"0" => 1.0, "1" => 1.0}), 1e-3)
assert_in_delta(0.5, t1.func([], {"0" => 1.0, "1" => 3.0}), 1e-3)
assert_in_delta(2.5, t1.func([], {"0" => 3.0, "1" => 1.0}), 1e-3)

## Question 1.2 (5 Points)

Implement the gradient function for $L(w)$. It evaluates the gradient for the value of $x$ for each dimension of $w$. In this simple case, $L(w)$ does not depend on $x$.

In [11]:
class ParabolaObjective
  def grad x, w
    dw = {"0" => (w["0"] - 1).to_f, "1" => (w["1"] - 2).to_f}
    return dw
  end
end

:grad

In [12]:
### TESTS ###
t2 = ParabolaObjective.new
w2_1 = t2.grad([], {"0" => 3.0, "1" => 7.0})
assert_in_delta(2.0, w2_1["0"], 1e-3)
assert_in_delta(5.0, w2_1["1"], 1e-3)

w2_2 = t2.grad([], {"0" => -3.0, "1" => -7.0})
assert_in_delta(-4.0, w2_2["0"], 1e-3)
assert_in_delta(-9.0, w2_2["1"], 1e-3)


## Question 2.1 (1 Point)


Implement gradient descent for any objective function class. Your function must provide a callback which we will use to monitor its performance and possibly to halt execution. A simple example illustrating the callback is as follows:

```ruby
def gradient_descent_example dataset, w, obj, learning_rate, tol, max_iter, &block
    iter = 1
    until converged(last_loss, current_loss) do
        w_last = w
        loss = calc_loss(w)
        w = update(w)
        w = adjust(w)
        iter += 1
        break unless yield w, iter, loss
    end
    
    return w
end

```

There are three main parts to the algorithm above:
1. Convergence is based on the absolute difference between the loss of the current and previoud iteration.
1. The norm can be calculated as the dot product of two vectors: $||w|| = w \cdot w$
1. Once the loss and gradient functions are calculated, we will update the values of each weight

### Implement dot product
In this first part, implement the dot product. The dot product below should be for sparse vectors. Use ```has_key?``` to skip entries in vector ```w``` not present in ```x```.

In [13]:
def dot x, w
  sum=0
  x.each do |key1, array1|
    w.each do |key2, array2|
      if key1==key2 then
        sum+=array1*array2
      end
    end
  end
  return sum
end


:dot

In [14]:
### TEST ###
assert_in_delta 6.0, dot({"a" => 2.0}, {"a" => 3.0}), 1e-6
assert_in_delta 6.0, dot({"a" => 2.0}, {"a" => 3.0, "b" => 4.0}), 1e-6
assert_equal 0.0, dot({}, {})
assert_equal 0.0, dot({"a" => 1.0}, {"b" => 1.0})

## Question 2.2 (1 Point)
Implement the L2 norm for a vector, i.e., $\left \lVert x \right \rVert$, when represented by a hash. Hint: use ```dot``` and don't forget the square root.

In [15]:
def norm w
  return dot(w,w)**(0.5)
end

:norm

In [16]:
### TEST ###
assert_in_delta 2.0, norm({"a" => 1.41421, "b" => 1.41421}), 1e-2
assert_in_delta 2.0, norm({"a" => -1.41421, "b" => 1.41421}), 1e-2
assert_in_delta 0.0, norm({}), 1e-2

## Question 2.3 (3 points)
Implement a function that updates a weight vector, ```w```, given a gradient vector, ```dw```, and learning rate, ```lr```.  Do not change the original weight vector. Hint: use ```clone```.

In [17]:
def update_weights(w, dw, lr)
  w2=w.clone
  w2.each do |key1, array1|
    dw.each do |key2, array2|
      if key1==key2 then
        w2[key1]=array1-dw[key2]*lr.to_f
      end
    end
  end
    return w2
end

:update_weights

In [18]:
### TEST ###
assert_in_delta 1.5, update_weights({"a" => 1.0}, {"a" => -0.25}, 2.0)["a"], 1e-2
assert_in_delta 2.5, update_weights({"a" => 1.0, "b" => 3.0}, {"a" => -0.25, "b" => 0.25}, 2.0)["b"], 1e-2

## Question 2.4 (15 Points)

Now, put all these functions together to implement gradient descent. This function takes the ```dataset``` and calls the function and gradient on all the examples. Hint: Increment ```iter``` before calling ```yield```.

In [20]:
def gradient_descent dataset, w, obj, learning_rate, tol, max_iter, &block  
  examples = dataset["data"]
  iter = 1
  lr=learning_rate
  loss_current=obj.func(dataset["data"], w.clone)
  
  until (loss_current<tol or max_iter<iter) do
    w_last = w.clone
    grad = obj.grad(dataset["data"], w.clone)
    w = update_weights(w.clone, grad, lr)
    w=obj.adjust(w)
    loss_current = obj.func(dataset["data"], w.clone)
    break unless yield w, iter, loss_current
    iter += 1
  end
  return w
end

:gradient_descent

In [21]:
### TESTS ###

t4 = ParabolaObjective.new
t4_w_init = {"0" => 3.0, "1" => 7.0}
t4_w_goal = {"0" => 1.0, "1" => 2.0}
t4_dataset = {"data" => []}

t4_loss = 1.0
t4_w = nil
gradient_descent t4_dataset, t4_w_init, t4, 0.1, 0.001, 100 do |w, iter, loss|
  t4_loss = loss
  t4_w = w
end

assert_in_delta 0.01, t4_loss, 1e-2
assert_in_delta 1.0, t4_w["0"], 1e-1
assert_in_delta 2.0, t4_w["1"], 1e-1

In [22]:
### TEST ###
t4 = ParabolaObjective.new
t4_w_init = {"0" => 3.0, "1" => 7.0}
t4_data = {"data" => []}

t4_total_loss = 0.0
t4_iterations = []
t4_losses = []
gradient_descent t4_data, t4_w_init, t4, 0.1, 0.001, 100 do |w, iter, loss|
  t4_total_loss += loss
  t4_iterations << iter
  assert_true(iter > 0, "Make sure to increment 'iter' before calling 'yield'")
  t4_losses << t4_total_loss / iter
end

assert_true(t4_iterations.size > 30)
assert_true(t4_losses[-1] < 3)
assert_true(t4_losses[-1] > 0)
assert_true(t4_iterations[-1] > 30)

In [23]:
### Plot the cumulative loss per iteration
Daru::DataFrame.new({x: t4_iterations, y: t4_losses}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Iteration"
  plot.y_label "Cumulative Loss"
end

#<Nyaplot::Plot:0x0000000002b08018 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0000000002b26680 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"6bad7bdd-290f-421f-8e53-7a4d307b4a9a"}, @xrange=[1, 46], @yrange=[1.3437385916027726, 11.745]>], :options=>{:x_label=>"Iteration", :y_label=>"Cumulative Loss", :zoom=>true, :width=>700, :xrange=>[1, 46], :yrange=>[1.3437385916027726, 11.745]}}>

## Question 3.1 (5 Points)

Let's learn the parameter of a Bernoulli distribution using the method of maximum likelihood. Consider the following dataset in which we are tossing a biased coin with probability $\mu$ of returning a success (1). There is an analytical solution to find this parameter $\mu$ given a dataset of successes and trials. Compute this analytical solution. 

Here the ```label``` field is either 0 or 1. 

### Dataset format
This is the format for the coin dataset. The format below will be used throughout most of the assignments. A dataset contains some extra details like the classes and names of features. The ```data``` entry is an array of examples containing ```features``` and a label. Notice that in this particular dataset, there aren't any "features", just a ```bias``` value.

In [98]:
puts JSON.pretty_generate(coin_dataset(1))

{
  "classes": {
  },
  "features": [
    "x"
  ],
  "data": [
    {
      "features": {
        "bias": 1.0
      },
      "label": 1.0
    }
  ]
}


In [24]:
coin_data = coin_dataset(1000)

def q31_binomial_parameter(coin_data)
  sum=0
  total=0
  coin_data["data"].each do |item|
    sum=sum+(item["label"]).to_f
    total=total+1
  end
  puts (sum.to_f)/(total.to_f)
  return (sum.to_f)/(total.to_f)
end

:q31_binomial_parameter

In [25]:
### TESTS ###

t31_coin_data = coin_dataset(10000)
assert_in_delta 0.77, q31_binomial_parameter(t31_coin_data), 5e-2

0.7691


## Question 3.2 (5 Points)

Now, let's use the maximum likelihood function and gradient descent to find the same parameter value. Define the objective function for a binomial distribution for multiple samples. Remember that the ```label``` is the target value and every example has only one feature, ```bias```. Learn the weight for the ```bias``` feature should converge to $w_{bias} = \mu$.

In this first step, calculate the log likelihood function for the binomial distribution of $n$ examples.

In [28]:
class BinomialModel
  def func examples, w
    count=0
    datasize=examples.size
    for i in 0..(datasize-1) do
      if examples[i]["label"]==1 then
        count+=1
      end
    end
    p=w["bias"]
    return ((-count)*Math.log(p)-(datasize-count)*Math.log(1-p))
  end
end

:func

In [29]:
### TEST ###
t32_model = BinomialModel.new
t32_t1 = {"features" => {"bias" => 1.0}, "label" => 0.0}
t32_t2 = {"features" => {"bias" => 1.0}, "label" => 1.0}
t32_dataset = {
  "data" => [t32_t1, t32_t2]
}

assert_in_delta 1.469677, t32_model.func([t32_t1], {"bias" => 0.77}), 1e-3
assert_in_delta 0.261365, t32_model.func([t32_t2], {"bias" => 0.77}), 1e-3
assert_in_delta 1.731041, t32_model.func(t32_dataset["data"], {"bias" => 0.77}), 1e-3


## Question 3.3 (5 Points)

Calculate the gradient of the binomial log likelihood function over $n$ examples.

In [30]:
class BinomialModel
  def grad examples, w
    g={}
    p=w["bias"]
    count=0
    datasize=examples.size
    for i in 0..(datasize-1) do
      if examples[i]["label"]==1 then
        count+=1
      end
    end
    g["bias"]=((-count)/(p)+(datasize-count)*(1/(1-p)))
    return g
  end
    def adjust w
    w["bias"] = [[0.001, w["bias"]].max, 0.999].min
    return w
  end
end

:adjust

In [31]:
### TEST ###
t33_model = BinomialModel.new
t33_t1 = {"features" => {"bias" => 1.0}, "label" => 0.0}
t33_t2 = {"features" => {"bias" => 1.0}, "label" => 1.0}

t33_dataset = {"data" => [t33_t1, t33_t2]}

assert_in_delta 4.347826, t32_model.grad([t33_t1], {"bias" => 0.77})["bias"], 1e-3
assert_in_delta -1.29870, t32_model.grad([t33_t2], {"bias" => 0.77})["bias"], 1e-3
assert_in_delta 3.049124, t32_model.grad(t33_dataset["data"], {"bias" => 0.77})["bias"], 1e-3

## Question 3.4 (5 Points)

Putting the objective function to work, use gradient descent to find the parameter for the binomial distribution. You get to set the learning rate. Return the learning rate you have obtained which works well. You may have to try a few until you get it right.

Note that, while capable of returning the same value, this method reads the data many more times than the analytical solution.

### Set the learning rate
Here, set this function to return one number. For example, if you decide that the learning rate needs to be 1.234, implement the following. Note: 1.234 might not be the best choice.

```ruby
def q34_learning_rate
  1.234
end
```

In [32]:
def q34_learning_rate
  return 0.00001
end

:q34_learning_rate

In [33]:
### TEST ###
t34 = BinomialModel.new
t34_w_init = {"bias" => rand}
t34_data = coin_dataset(10000)

t34_learning_rate = q34_learning_rate()

t34_total_loss = 0.0
t34_iterations = []
t34_losses = []
t34_w = t34_w_init
gradient_descent t34_data, t34_w_init, t34, t34_learning_rate, 0.001, 100 do |w, iter, loss|
  puts [iter, w, t34_losses[-1]].join("\t") if iter % 10 == 1
  t34_total_loss += loss
  t34_iterations << iter
  assert_true(iter > 0, "Make sure to increment iter before calling yield")
  t34_losses << t34_total_loss / iter.to_f
  t34_w = w
end


assert_true(t34_losses[-1] < 8000)
assert_true(t34_losses[-1] > 0)
assert_true(t34_iterations[-1] > 30)
assert_in_delta 0.77, t34_w["bias"], 5e-2

### Plot the cumulative loss per iteration
Daru::DataFrame.new({x: t34_iterations, y: t34_losses}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Iteration"
  plot.y_label "Cumulative Loss"
end

1	{"bias"=>0.5866049508927114}	
11	{"bias"=>0.7804134998199271}	5391.81340478645
21	{"bias"=>0.7804999864688038}	5327.27871264938
31	{"bias"=>0.7804999999978848}	5305.767139798877
41	{"bias"=>0.7804999999999996}	5295.011353373624
51	{"bias"=>0.7805}	5288.557881518474
61	{"bias"=>0.7805}	5284.255566948372
71	{"bias"=>0.7805}	5281.182485112586
81	{"bias"=>0.7805}	5278.877673735746
91	{"bias"=>0.7805}	5277.085042664869


#<Nyaplot::Plot:0x0000000002b9dca8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000000029e9808 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"33a0630e-2e93-4bf9-9d1e-0be6a362d33c"}, @xrange=[1, 100], @yrange=[5275.65093780817, 6102.172521073473]>], :options=>{:x_label=>"Iteration", :y_label=>"Cumulative Loss", :zoom=>true, :width=>700, :xrange=>[1, 100], :yrange=>[5275.65093780817, 6102.172521073473]}}>

## Question 4.1 (10 Points)

The maximum likelihood method above reads the data multiple times and can benefit from prior knowledge in the form of a prior distribution for the parameter, $\mu$. Using the Beta distribution as the conjugate prior, implement the likelihood function and its gradient. Now we are learning three parameters altogether: $w_{bias} = \mu$, $\alpha$, $\beta$.

First, let's compute the closed form estimator for $\mu$ with a fixed $\alpha$ and $\beta$.

In [34]:
def q41_closed_form_beta_binomial(coin_data, alpha, beta)
  return (alpha-1)/(alpha+beta-2).to_f
end

:q41_closed_form_beta_binomial

In [35]:
### TEST ###
t41_coin_data = coin_dataset(10000)
assert_in_delta 0.77, q41_closed_form_beta_binomial(t41_coin_data, 7743, 10000 - 7743), 1e-1

## Question 4.2 (10 points)

Implement the negative log likelihood function for the beta + binomial values. Checkout this [Wikipedia](https://en.wikipedia.org/wiki/Beta_distribution#Maximum_likelihood) definition of the likelihood function. Remember we are interested in he negative log likelihood.

A special function is needed ```GSL::Sf::lnbeta(a, b)```.

In [36]:
class BetaBinomialModel
  def func dataset, w
    sum1=0.0
    sum2=0.0
    alpha=w["alpha"]
    beta=w["beta"]
    adjust(w)
    p=w["bias"]
    size=dataset.size
    count=0
    
    for i in 0..(size-1) do
      if dataset[i]["label"]==1.0
        count+=1
      end
    end
    
    for i in 0..(size-1) do
      sum1=sum1+Math.log(p)
    end
    
    for i in 0..(size-1) do
      sum2=sum2+Math.log((1-p))
    end
    
    a = ((alpha-1)*(size)+count)*Math.log(p)
    b = ((beta-1)*(size)+size-count)*Math.log(1-p)
    c = -1*GSL::Sf::lnbeta(alpha, beta)*(size)

    answer=a+b+c

    return -answer
  end
  
  def adjust w
    w["bias"] = [[0.001, w["bias"]].max, 0.999].min
    w["beta"] = [0.0001, w["beta"]].max
    return w
  end
end

:adjust

In [37]:
### TESTS ###

t42 = BetaBinomialModel.new
srand 777 #seed random number generator
t42_data = coin_dataset(1000)["data"]
t42_w = Hash.new {|h,k| h[k] = 0.1}
t42_w["alpha"] = 7.0
t42_w["beta"] = 3.0

assert_in_delta 10373.126026759332, t42.func(t42_data, t42_w), 1e-2

## Question 4.3 (10 points)

Implement the negative log likelihood gradient for all the parameters. Checkout this [Wikipedia](https://en.wikipedia.org/wiki/Beta_distribution#Maximum_likelihood) definition of the likelihood function. Remember we are interested in the negative log likelihood. The gradient for the ```bias``` requires all examples in the dataset. However, the gradient for ```alpha``` and ```beta``` does not require a pass over the dataset.

A special function is needed ```GSL::Sf::psi(a + b)```.

In [38]:
class BetaBinomialModel
  def grad dataset, w
    sum1=0.0
    sum2=0.0
    alpha=w["alpha"]
    beta=w["beta"]
    p=w["bias"]
    size=dataset.size
    grad={}

    count=0

    for i in 0..(size-1) do
      if dataset[i]["label"]==1.0
        count+=1
      end
    end

    a=((alpha-1)+count)/p
    b=((beta-1)+size-count)/(1-p)
    grad["bias"]=-a+b
    
    grad["alpha"]=-(Math.log(p)-(-GSL::Sf::psi(alpha + beta)+GSL::Sf::psi(alpha)))
    
    grad["beta"]=-(Math.log(1-p)-(-GSL::Sf::psi(alpha + beta)+GSL::Sf::psi(beta)))

    return grad

  end
end

:grad

In [39]:
### TESTS ###

t43 = BetaBinomialModel.new
srand 777 #seed random number generator
t43_data = coin_dataset(1000)["data"]
t43_w = Hash.new {|h,k| h[k] = 0.1}
t43_w["alpha"] = 7.0
t43_w["beta"] = 3.0

t43_grad = t43.grad(t43_data, t43_w)

assert_in_delta -7902.2222222221935, t43_grad["bias"], 1e2
assert_in_delta 1.9236168390257913, t43_grad["alpha"], 1e-1
assert_in_delta -1.2236077383104214, t43_grad["beta"], 1e-1


## Question 4.4 (20 points)

Run the gradient descent by selecting the initial weights and learning rate. Try a few values. 

In [40]:
def q44_weights_and_learning_rate
  w={}
  w["alpha"]=0.7
  w["beta"]=0.455
  w["bias"]=0.77
  lr=0.000001
  return [w, lr]
end

:q44_weights_and_learning_rate

In [41]:
### TEST ###
t44 = BetaBinomialModel.new
t44_data = coin_dataset(10000)

t44_w_init, t44_learning_rate = q44_weights_and_learning_rate()

t44_total_loss = 0.0
t44_iterations = []
t44_losses = []
t44_w = t34_w_init
gradient_descent t44_data, t44_w_init, t44, t44_learning_rate, 0.001, 100 do |w, iter, loss|
  puts [iter, w, t44_losses[-1]].join("\t") if iter % 10 == 1
  t44_total_loss += loss
  t44_iterations << iter
  t44_losses << t44_total_loss / iter.to_f
  t44_w = w
end


puts "it's one"
puts t44_losses
assert_true(t44_losses[-1] < 8000)
puts "it's two"
puts t44_iterations
assert_true(t44_iterations[-1] > 30)
assert_in_delta 0.77, t44_w["bias"], 5e-2

### Plot the cumulative loss per iteration
Daru::DataFrame.new({x: t44_iterations, y: t44_losses}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Iteration"
  plot.y_label "Cumulative Loss"
end

1	{"alpha"=>0.7000006110393128, "beta"=>0.45500038694404443, "bias"=>0.7704367625635234}	
11	{"alpha"=>0.7000067479083389, "beta"=>0.45500416699663254, "bias"=>0.7736692085416402}	6443.590568736936
21	{"alpha"=>0.7000129174250792, "beta"=>0.45500783552710217, "bias"=>0.7754808824683709}	6417.776278484134
31	{"alpha"=>0.7000191051119424, "beta"=>0.45501144116569014, "bias"=>0.7764888258471395}	6399.497710137873
41	{"alpha"=>0.7000253028539046, "beta"=>0.45501501167041286, "bias"=>0.7770472549621588}	6386.311325335448
51	{"alpha"=>0.7000315061358376, "beta"=>0.45501856264284557, "bias"=>0.7773559105125312}	6376.599992111794
61	{"alpha"=>0.7000377124561584, "beta"=>0.45502210277568567, "bias"=>0.7775262864753067}	6369.292815112673
71	{"alpha"=>0.7000439204321666, "beta"=>0.455025636888613, "bias"=>0.7776202642974038}	6363.675958845294
81	{"alpha"=>0.7000501293006134, "beta"=>0.4550291676466597, "bias"=>0.7776720806536951}	6359.268614408228
91	{"alpha"=>0.7000563386404962, "beta"=>0.455032

#<Nyaplot::Plot:0x00000000031decc0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000000031a5808 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"4807ae74-b489-45e8-b1aa-184e25e2de69"}, @xrange=[1, 100], @yrange=[6352.871086570417, 6476.25921835393]>], :options=>{:x_label=>"Iteration", :y_label=>"Cumulative Loss", :zoom=>true, :width=>700, :xrange=>[1, 100], :yrange=>[6352.871086570417, 6476.25921835393]}}>

In [42]:
### TEST ###
t44 = BetaBinomialModel.new
t44_data = coin_dataset(10000)

t44_w_init, t44_learning_rate = q44_weights_and_learning_rate()

t44_total_loss = 0.0
t44_iterations = []
t44_losses = []
t44_w = t34_w_init
gradient_descent t44_data, t44_w_init, t44, t44_learning_rate, 0.001, 100 do |w, iter, loss|
  puts [iter, w, t44_losses[-1]].join("\t") if iter % 10 == 1
  t44_total_loss += loss
  t44_iterations << iter
  t44_losses << t44_total_loss / iter.to_f
  t44_w = w
end


assert_true(t44_losses[-1] < 8000)
assert_true(t44_iterations[-1] > 30)
assert_in_delta 0.77, t44_w["bias"], 5e-2

### Plot the cumulative loss per iteration
Daru::DataFrame.new({x: t44_iterations, y: t44_losses}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Iteration"
  plot.y_label "Cumulative Loss"
end

1	{"alpha"=>0.7000006110393128, "beta"=>0.45500038694404443, "bias"=>0.7704762882552231}	
11	{"alpha"=>0.7000067503096319, "beta"=>0.4550041588379397, "bias"=>0.7740028014066637}	6431.263958176182
21	{"alpha"=>0.7000129251934665, "beta"=>0.45500780889791337, "bias"=>0.7759796975963831}	6403.022259586012
31	{"alpha"=>0.7000191198929929, "beta"=>0.45501139020421055, "bias"=>0.7770790362947435}	6383.021195344271
41	{"alpha"=>0.7000253255511569, "beta"=>0.4550149331130293, "bias"=>0.7776875645421539}	6368.593604862423
51	{"alpha"=>0.7000315372422828, "beta"=>0.4550184546943021, "bias"=>0.778023538443271}	6357.970641045892
61	{"alpha"=>0.7000377522397137, "beta"=>0.4550219644554369, "bias"=>0.7782087652458693}	6349.979780118316
71	{"alpha"=>0.7000439690380085, "beta"=>0.45502546766349655, "bias"=>0.7783108015906552}	6343.839170155395
81	{"alpha"=>0.70005018680719, "beta"=>0.4550289672277521, "bias"=>0.7783669857646873}	6339.022190797245
91	{"alpha"=>0.7000564050900435, "beta"=>0.45503246475

#<Nyaplot::Plot:0x0000000003269a00 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000000032303e0 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"489ded59-5088-4d49-86d4-b78ae5399634"}, @xrange=[1, 100], @yrange=[6332.032638275235, 6466.981467387201]>], :options=>{:x_label=>"Iteration", :y_label=>"Cumulative Loss", :zoom=>true, :width=>700, :xrange=>[1, 100], :yrange=>[6332.032638275235, 6466.981467387201]}}>